# E6: Recursive NN

Experimental Algorithm:
- 57 LFs -> 20 randomly sampled sets of 5 -> Snorkel -> 20 LFs -> analysis
- 20 LFs -> 20 randomly sampled sets of 5 -> Snorkel -> 20 LFs -> analysis
- repeat

In [ ]:
from snorkel.labeling.model.label_model import LabelModel
from sklearn.model_selection import train_test_split
from Our_Monitors.CD_Monitor import CDM
from Our_Monitors.CDGA_Monitor import CDGAM
import numpy as np
import logging

In [ ]:
# Parameters for the analysis
num_iters = 10
num_subsets = 20
subset_size = 5
with_replacement = True
lf_subset = list(range(57))
metrics = ["accuracy","f1"]

In [ ]:
# Using a subset of the LFs in the analysis
L_data = np.copy(L_alarms[:,lf_subset])
Y_data = alarms_df.true_label.values

In [ ]:
# set seed as current time to get different results
np.random.seed(int(time()))

In [ ]:
all_scores = []
all_deps = []

for iter in range(num_iters):
    logging.info("-- Iteration ", iter + 1, "--")

    # Randomly sample subsets of specified size
    try:
        subsets = np.random.choice(L_data.shape[1], size=(num_subsets,subset_size), replace=with_replacement)
    except ValueError:
        print("Cannot perform sample...")
        break

    # Define a new LF per subset as a Snorkel model over the LFs in the subset
    new_L_data = np.zeros((L_data.shape[0], num_subsets))
    
    for i, subset in enumerate(subsets):
        l_model = LabelModel(cardinality=2, verbose=True)
        l_model.fit(L_data[:,subset])
        new_L_data[:,i] = l_model.predict(L_data[:,subset])

    L_data = np.copy(new_L_data)

    # Evaluate model trained over new LFs
    l_model = LabelModel(cardinality=2, verbose=True)
    l_model.fit(L_data)
    scores = l_model.score(L_data, Y_data, metrics=metrics)
    logging.info(scores)
    all_scores.append(scores)

    # Find dependencies between new LFs
    L_train, L_dev, Y_train, Y_dev = train_test_split(L_data, Y_data, test_size=0.2, shuffle=True)
    #deps = CDM(L_dev, Y_dev, k=2, sig=0.05, policy="old", verbose=False)
    deps = CDGAM(L_dev, k=2, sig=0.05, policy="new", verbose=False, return_more_info=False)
    print(len(deps))
    all_deps.append(len(deps))